In [2]:
#!pip install beautifulsoup4


   ---------------------------------------- 0.0/147.9 kB ? eta -:--:--
   -- ------------------------------------- 10.2/147.9 kB ? eta -:--:--
   -------------------------------------- - 143.4/147.9 kB 2.1 MB/s eta 0:00:01
   ---------------------------------------- 147.9/147.9 kB 2.2 MB/s eta 0:00:00


In [3]:
import requests
from bs4 import BeautifulSoup

In [11]:
infectiousdiseases = 'https://medlineplus.gov/infections.html'

try:
    response = requests.get(url)
    response.raise_for_status()  # Lança uma exceção se ocorrer um erro HTTP
except requests.exceptions.RequestException as e:
    print('Erro ao fazer a requisição:', e)
    exit()

infectiousdiseases_html = BeautifulSoup(response.content, 'html.parser')

In [13]:
# Salvar o conteúdo HTML em um arquivo
infectiousdiseases = str(infectiousdiseases_html)
with open('infectiousdiseases.html', 'w', encoding='utf-8') as file:
    file.write(infectiousdiseases)

print('HTML salvo com sucesso no arquivo "pagina.html"')

HTML salvo com sucesso no arquivo "pagina.html"


In [19]:
# Carregar o arquivo HTML local
with open('infectiousdiseases.html', 'r', encoding='utf-8') as file:
    infectiousdiseases_reader = file.read()

# Criar um objeto BeautifulSoup
infectiousdiseases_soup = BeautifulSoup(infectiousdiseases_reader, 'html.parser')


In [29]:
# Encontrar todos os elementos com a classe "item"
items = infectiousdiseases_soup.find_all(class_='item')

# Inicializar uma lista para armazenar as doenças
diseases = []

# Exibir os textos dos elementos <a> dentro dos elementos com a classe "item"
for item in items:
    # Encontrar todos os elementos <a> dentro do elemento com a classe "item"
    a_tags = item.find_all('a')
    for a_tag in a_tags:
        disease = a_tag.text.strip()
        if disease != 'Animal Diseases and Your Health':
            diseases.append(disease)



In [30]:

# Contar o número total de doenças
total_diseases = len(diseases)

# Exibir as 5 primeiras e as 5 últimas doenças
print(f"Total de doenças: {total_diseases}")
print("\n5 primeiras doenças:")
for disease in diseases[:5]:
    print(disease)
print("\n5 últimas doenças:")
for disease in diseases[-5:]:
    print(disease)

Total de doenças: 280

5 primeiras doenças:
Aspergillosis
Abscess
HIV
Acute Bronchitis
Acute Flaccid Myelitis

5 últimas doenças:
Whooping Cough
West Nile Virus
Yeast Infections
Hemorrhagic Fevers
Zika Virus


In [33]:
disease = f'https://medlineplus.gov/{diseases[0].lower()}.html'
try:
    response = requests.get(url)
    response.raise_for_status()  # Lança uma exceção se ocorrer um erro HTTP
except requests.exceptions.RequestException as e:
    print('Erro ao fazer a requisição:', e)
    exit()

disease_html = BeautifulSoup(response.content, 'html.parser')
disease_html

<!DOCTYPE html>

<html class="nojs us" data-root="https://medlineplus.gov/" id="health_topics_by_group" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
<meta content="_top" http-equiv="window-target"/>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="https://medlineplus.gov/infections.html" rel="canonical"/>
<link href="https://medlineplus.gov/infections.html" hreflang="en" rel="alternate"/>
<link href="https://medlineplus.gov/spanish/infections.html" hreflang="es" rel="alternate"/>
<meta content="medlineplus-ac-dictionary" name="ac-dictionary"/>
<link href="http://purl.org/dc/elements/1.1/" rel="schema.DC" title="The Dublin Core metadata Element Set"/>
<meta content="Infections" name="keywords"/>
<meta content="Infections" name="DC.Relation.IsPartOf"/>
<meta content="Communicable Diseases" name="DC.Subject.MeSH">
<meta content=